<h3> Loading the libraries <h3>

In [28]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json
import pandas as pd


<h3> Read the data <h3>

In [29]:
data =  pd.read_csv('wtb1.csv')
print(data.head())

                  Domain name            Company name Twitter  \
0         chateau-giscours.fr        Château Giscours     NaN   
1                    raimo.fr           Raimo Glacier     NaN   
2  champagne-voirin-jumel.com  Champagne Voirin Jumel     NaN   
3        chateaudepoisses.com      Château d'Époisses     NaN   
4              parenchere.com   Château de Parenchère     NaN   

              Facebook  Unnamed: 0  \
0                  NaN           0   
1         raimoglacier           1   
2                  NaN           2   
3     chateaudepoisses           3   
4  chateaudeparenchere           4   

                                  lighthouse_metrics  \
0  {'mobile': {'performance': 0.59, 'accessibilit...   
1  {'mobile': {'performance': 0.61, 'accessibilit...   
2  {'mobile': {'performance': 0.39, 'accessibilit...   
3  {'mobile': {'performance': 0.32, 'accessibilit...   
4  {'mobile': {'performance': 0.35, 'accessibilit...   

                                     contac

<h3> Inspecting the data <h3>

In [30]:
data.info()
data.describe()
data.isnull().sum()
data.duplicated().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2022 entries, 0 to 2021
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Domain name         2022 non-null   object
 1   Company name        1940 non-null   object
 2   Twitter             446 non-null    object
 3   Facebook            1288 non-null   object
 4   Unnamed: 0          2022 non-null   int64 
 5   lighthouse_metrics  2022 non-null   object
 6   contact_details     2022 non-null   object
dtypes: int64(1), object(6)
memory usage: 110.7+ KB


0

<h3> Fill missing values <h3>

In [31]:

data['Twitter'].fillna('unknown', inplace=True)
data['Facebook'].fillna('unknown', inplace=True)


/var/folders/h4/kdky5skx0jggltns31j4zj7r0000gn/T/ipykernel_21762/1236659601.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Twitter'].fillna('unknown', inplace=True)
/var/folders/h4/kdky5skx0jggltns31j4zj7r0000gn/T/ipykernel_21762/1236659601.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

<h3> Remove duplicates <h3>

In [32]:
data.drop_duplicates(inplace=True)

<h3> Validate domain names <h3>

In [33]:
import re
def validate_domain(domain):
    pattern = re.compile(r'^(?:[a-z0-9](?:[a-z0-9-]{0,61}[a-z0-9])?\.)+[a-z]{2,6}$')
    return bool(pattern.match(domain))
data['Domain_valid'] = data['Domain name'].apply(validate_domain)


<h3> Validate social media  names <h3>

In [34]:
def validate_twitter(handle):
    pattern = re.compile(r'^@?(\w){1,15}$')
    return bool(pattern.match(handle))

def validate_facebook(handle):
    pattern = re.compile(r'^[a-z\d.]{5,}$')
    return bool(pattern.match(handle))

data['Twitter_valid'] = data['Twitter'].apply(validate_twitter)
data['Facebook_valid'] = data['Facebook'].apply(validate_facebook)





In [35]:
data.to_csv('wtb-internship-test.csv')